# Example use of a schedview visit table in a Times Square notebook

In [ ]:
# Times Square parameters

night = '2024-11-15'
visit_origin = 'lsstcomcam'

In [ ]:
# Use more recent versions of rubin_scheduler and schedview (will become unnecessary soon)
import sys
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-11-21/rubin_scheduler')
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-11-21/schedview')


# Imports
import bokeh.io
import astropy.utils.iers
from schedview import DayObs
from schedview.collect.visits import read_visits
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from schedview.compute.visits import add_coords_tuple
from schedview.plot.visitmap import create_visit_skymaps

# Tell astropy we don't need super high precision:
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Tell rubin_scheduler where to find its data
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Prepare jupyter to show bokeh plots
bokeh.io.output_notebook()

In [ ]:
day_obs = DayObs.from_date(night)

In [ ]:
visits = read_visits(day_obs, visit_origin)

In [ ]:
observatory = ModelObservatory(init_load_length=1)
observatory.mjd = visits.observationStartMJD.median()

In [ ]:
visits = add_coords_tuple(visits)
p, vmap_data = create_visit_skymaps(
    visits=visits,
    night_date=day_obs.date,
    timezone="Chile/Continental",
    observatory=observatory,
)
bokeh.io.show(p)


The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The "Armillary sphere" shows the sphere in orthographic projection, with the center point of the projection controlled by the "center alt" and "center az" sliders beneath the plot. A static orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The "Planisphere" shows the sky in [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole, with R.A. increasing counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan. The fill colors are set according to the [descolors palette](https://github.com/DarkEnergySurvey/descolors):

 - <span style='background-color:#56b4e9'>&nbsp;&nbsp;&nbsp;</span><span style='color:#56b4e9'> blue</span>: u band
 - <span style='background-color:#008060'>&nbsp;&nbsp;&nbsp;</span><span style='color:#008060'> green</span>: g band
 - <span style='background-color:#ff4000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ff4000'> red</span>: r band
 - <span style='background-color:#850000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#850000'> brown/crimson</span>: i band
 - <span style='background-color:#6600cc'>&nbsp;&nbsp;&nbsp;</span><span style='color:#6600cc'> purple</span>: z band
 - <span style='background-color:#000000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#000000'> black</span>: y band

Both plots have the following additional annotations:
 - The gray background shows the planned final depth of the LSST survey.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) , completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.